# Introdução

Agora é com você!. Use o que desenvolvemos na prática guiada para aplicar o grid search em novos dados.


Diretrizes:<br>
    1) Use o arquivo de diabetes.csv para criar um modelo KNN<br>
    2) Observe a documentação da class KNeighborsClassifier e veja os hiperparâmetros <br>disponíveis e quais seus possíveis valores. Escolha 3 hiperparâmetros para otimizar.<br>
    3) Crie um Pipeline que combina ```MinMaxScaler``` e o ```KNeighborsClassifier```<br> 
    4) Crie uma grid para guardar os ranges de valores que você criou  para cada hiperparâmetro <br>
    5) Defina uma validação <br>
    6) Aplique o ```GridSearchCV```<br>
    7) Avalie a performance no treino e no teste.<br>

# Setup

In [ ]:
# Leia seus dados e carregue todas as classes e funções que vai utilizar 

# Pipeline e Grid

In [1]:
# Crie o pipeline aqui

In [3]:
# Defina a grid de hiperparâmetros. 
# Não esqueça de adicionar "prefixo__", 
# sendo o prefixo o nome que você dá ao step que contém o modelo dentro do pipeline.

# Grid Search

In [ ]:
# Defina o método de validação,
# defina a classe do Grid Search e inicialize a busca

# Performance

In [ ]:
# Avalie a performance no treino e no teste

# Avançado

Proponha alguma alteração no modelo, dada a sua curva de validação.